<a href="https://colab.research.google.com/github/TapasKumarDutta1/IEEE-CIS-Fraud/blob/master/Cleaning_Preprocessingand_adding_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting data

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


Loading drive

In [ ]:
import warnings
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder

# Mount Google Drive for data access
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


Reading datasets

In [ ]:
# Read the train_transaction.csv.zip file into the 'trn' DataFrame
trn = pd.read_csv("train_transaction.csv.zip")

# Extract the 'isFraud' column from 'trn' and store it separately in 'isFraud' variable
isFraud = trn["isFraud"]

# Drop the 'isFraud' column from the 'trn' DataFrame
trn = trn.drop(["isFraud"], axis=1)

# Read the test_transaction.csv.zip file into the 'tst' DataFrame
tst = pd.read_csv("test_transaction.csv.zip")


Feature selection

In [ ]:
# Extract the names of useful columns (excluding "TransactionID") from the train dataset
useful_cols = list(trn.iloc[:, :55])
useful_cols.remove("TransactionID")

# Keep only the useful columns in the train dataset
trn = trn[useful_cols]

# Keep only the useful columns in the test dataset
tst = tst[useful_cols]

Creating unique id

In [ ]:
# Calculate the day from the "TransactionDT" in seconds and assign it to a new column "day"
trn["day"] = trn["TransactionDT"] // 86400
tst["day"] = tst["TransactionDT"] // 86400

# Create new empty columns "id" in both train and test datasets
trn["id"] = ""
tst["id"] = ""

# Calculate the difference "d_1" by subtracting the value in column "D1" from the "day"
trn["d_1"] = trn["day"] - trn["D1"]
tst["d_1"] = tst["day"] - tst["D1"]

# Concatenate selected columns to create the new "id" for train and test datasets
for col in ["C1", "C13", "P_emaildomain", "addr1", "card1", "d_1"]:
    trn["id"] += trn[col].astype(str)
    tst["id"] += tst[col].astype(str)

# Store the "id" columns in separate variables for train and test datasets
id_trn = trn["id"]
id_tst = tst["id"]

seperate columns in continuous and categorical

In [ ]:
# List of categorical features
categorical = [
    "ProductCD",
    "card1",
    "card2",
    "card3",
    "card4",
    "card5",
    "card6",
    "addr1",
    "addr2",
    "P_emaildomain",
    "R_emaildomain",
] + ["M" + str(i) for i in range(1, 10)]

# List of continuous features
continuous = [i for i in useful_cols if i not in categorical]

Preprocessing

In [ ]:
class ContinuousConverter:
    def __init__(self, name, log_transform=False):
        """
        Initialize a ContinuousConverter instance.

        Parameters:
            name (str): The name of the continuous feature.
            log_transform (bool, optional): Whether to apply log-transform on the feature. Default is False.
        """
        self.name = name
        self.skew = None
        self.log_transform = log_transform

    def transform(self, feature):
        """
        Transform the continuous feature.

        Parameters:
            feature (pd.Series): The continuous feature to be transformed.

        Returns:
            pd.Series: Transformed continuous feature.
        """
        if self.log_transform:
            feature = self._apply_log_transform(feature)

        mean = feature.mean()
        std = feature.std()
        feature = (feature - mean) / (std + 1e-6)
        return feature

In [ ]:
continuous_trn = []
continuous_tst = []

# Iterate through continuous features and perform transformations
for col in tqdm(continuous):
    # Define the log transformation function
    log = lambda x: np.log10(x + (1 - min(0, min(x))))
    # Initialize the ContinuousConverter object
    converter = ContinuousConverter(col, log, trn[col])
    # Apply the transformation on training data and append to continuous_trn list
    continuous_trn.append(converter.transform(trn[col]).astype(np.float32))
    # Apply the transformation on test data and append to continuous_tst list
    continuous_tst.append(converter.transform(tst[col]).astype(np.float32))

# Create pandas DataFrame from the transformed lists
continuous_trn = pd.DataFrame(continuous_trn).T
continuous_trn.columns = continuous
continuous_tst = pd.DataFrame(continuous_tst).T
continuous_tst.columns = continuous

100%|██████████| 34/34 [00:04<00:00,  7.40it/s]


In [ ]:
# Calculate the sum of NaN values for each row in 'continuous_trn' DataFrame and add a new column 'isna_sum'
continuous_trn["isna_sum"] = continuous_trn.isna().sum(axis=1)

# Calculate the sum of NaN values for each row in 'continuous_tst' DataFrame and add a new column 'isna_sum'
continuous_tst["isna_sum"] = continuous_tst.isna().sum(axis=1)

# Calculate the mean and standard deviation of 'isna_sum' column in 'continuous_trn' DataFrame
mn = continuous_trn["isna_sum"].mean()
std = continuous_trn["isna_sum"].std()

# Normalize the 'isna_sum' column in 'continuous_trn' DataFrame
continuous_trn["isna_sum"] = (continuous_trn["isna_sum"] - mn) / std

# Calculate the mean and standard deviation of 'isna_sum' column in 'continuous_tst' DataFrame
mn = continuous_tst["isna_sum"].mean()
std = continuous_tst["isna_sum"].std()

# Normalize the 'isna_sum' column in 'continuous_tst' DataFrame
continuous_tst["isna_sum"] = (continuous_tst["isna_sum"] - mn) / std

# Create a list 'isna_cols' to store column names with missing values
isna_cols = []

# Loop through each column in 'continuous' list
for col in continuous:
    # Check if the column has any missing values
    isna = continuous_trn[col].isna()
    if isna.sum() > 0:
        # Add a new binary column to indicate whether the original column has missing values
        continuous_trn[col + "_isna"] = isna.astype(int)
        continuous_tst[col + "_isna"] = continuous_tst[col].isna().astype(int)

# Fill missing values in 'continuous_trn' and 'continuous_tst' DataFrames with 0
continuous_trn = continuous_trn.fillna(0)
continuous_tst = continuous_tst.fillna(0)

Handling categorical data

In [ ]:
def handle_categorical(df_trn, df_tst, categorical, n_values):
    """
    Encode categorical features using one-hot encoding and handle rare categories.

    Parameters:
        df_trn (pd.DataFrame): Training data DataFrame.
        df_tst (pd.DataFrame): Test data DataFrame.
        categorical (list): List of categorical feature names to be encoded.
        n_values (int): Number of top frequent values to keep for each categorical feature.

    Returns:
        pd.DataFrame, pd.DataFrame: Encoded training and test DataFrames.
    """
    all_categories = []
    df_trn = df_trn[categorical]
    df_tst = df_tst[categorical]

    for col in categorical:
        # Get top frequent values and "Other" for remaining values
        all_categories.append(list(df_trn[col].value_counts().iloc[: n_values - 1].index) + ["Other"])

        # Map values to top frequent values or "Other"
        df_trn[col] = df_trn[col].map(lambda x: x if x in all_categories[-1] else "Other")
        df_tst[col] = df_tst[col].map(lambda x: x if x in all_categories[-1] else "Other")

    # One-hot encoding using top frequent categories for each feature
    ohe = OneHotEncoder(categories=all_categories)
    ohe.fit(pd.concat([df_trn, df_tst]))

    # Transform the data into one-hot encoded DataFrame
    df_trn_encoded = pd.DataFrame(ohe.transform(df_trn).toarray()).astype(np.float16)
    df_tst_encoded = pd.DataFrame(ohe.transform(df_tst).toarray()).astype(np.float16)

    return df_trn_encoded, df_tst_encoded

# Example usage
# Assuming 'trn' and 'tst' are already defined DataFrames
categorical_features = [...]  # List of categorical feature names
num_top_values = 50  # Number of top frequent values to keep for each categorical feature
df_trn_encoded, df_tst_encoded = handle_categorical(trn, tst, categorical_features, num_top_values)

print("Encoded Training Data Shape:", df_trn_encoded.shape)
print("Encoded Test Data Shape:", df_tst_encoded.shape)

((590540, 444), (506691, 444))

Concatenating continuous and categorical

In [ ]:
# Concatenate 'df_trn' and 'continuous_trn' DataFrames horizontally (axis=1)
trn = pd.concat([df_trn, continuous_trn], axis=1)

# Concatenate 'df_tst' and 'continuous_tst' DataFrames horizontally (axis=1)
tst = pd.concat([df_tst, continuous_tst], axis=1)

Columns created

In [ ]:

# Add 'id', 'isFraud' columns to the DataFrames
trn["id"] = id_trn
tst["id"] = id_tst
trn["isFraud"] = isFraud

# Save DataFrames to CSV files
tst.to_csv("/content/gdrive/My Drive/fraud/test.csv", index=False)  # Set index=False to exclude the index column
trn.to_csv("/content/gdrive/My Drive/fraud/train.csv", index=False)